# Atoti vs pandas Comparative Analysis for Value at Risk: pandas Notebook

To compare the advanced analytics capabilities of Atoti with that of pandas, we will build a notebook that implements, **using pandas**, the main components of a VaR use case:

* Compute VaR and ES:
    * At two different confidence levels: 95% and 99%
    * At three different granularities: top-of-house (global value for entire financial institution), combination of book and trade, and combination of all attributes
* Track, for each of those queries, the:
    * Response time
    * Memory usage
* We will also enrich the use case by:
    * Computing the marginal VaR
    * Performing simulations

💡 **Note:** This notebook downloads a dataset of ~5.5GB from Amazon S3, which may take some time to initially load depending on internet speed.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=atoti-pandas-comparison" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Discover-Atoti-now.png" alt="Try Atoti"></a></div>

In [1]:
%load_ext memory_profiler

In [2]:
import pandas as pd
import numpy as np
import s3fs
from utils import progress_bar

## Loading the data into a pandas dataframe

### Trade PnL Table

Our trade table is the main table that contains all trades, attributes, and most importantly, the Profit-and-Loss (PnL) vectors.

We will define the structure of the table, then feed it from our S3 repository. This will be a rather large dataset, which will allow us to perform our comparative analysis at a significant level.

In [3]:
# Use anonymous access with s3fs (required for accessing a public S3 bucket)
s3fs.S3FileSystem.read_timeout = 1200
s3 = s3fs.S3FileSystem(anon=True)
objects = s3.ls("s3://data.atoti.io/notebooks/atoti-pandas-comparison/data/pnl")

# Get total number of files for progress bar calculation and initialize counter
total_files = sum(1 for obj in objects if obj.endswith(".parquet"))
file_count = 1

# Iterate over files in the S3 bucket and display the progress bar
dataframes = []
for obj in objects:
    file = f"s3://{obj}"
    if file != "s3://data.atoti.io/notebooks/atoti-pandas-comparison/data/pnl/":
        progress_bar.print_progress_bar(
            file_count,
            total_files,
            prefix=f"Loading {file_count} / {total_files} files",
            suffix="Complete",
            length=50,
        )
        temp_df = pd.read_parquet(file, filesystem=s3)
        dataframes.append(temp_df)
        file_count += 1

trades = pd.concat(dataframes, ignore_index=True)
trades.head()

Loading 31 / 31 files |██████████████████████████████████████████████████| 100.0% Complete

,BOOKID,ASOFDATE,TRADEID,DATASET,RISKFACTOR,RISKCLASS,SENSITIVITYNAME,CCY,TID,PNL_VECTOR
0,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,Credit spread_Allegheny Energy_Implied spread,Credit spread,delta,EUR,7454,"[-1277.1016253973248, -170.05582820769533, 587..."
1,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,Foreign exchange_USD_Spot price,Foreign exchange,delta,EUR,7454,"[26194.603468126214, -15146.454845596281, 2217..."
2,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,Interest rate_USD.OIS_Implied yield,Interest rate,delta,EUR,7454,"[-125.09660412815069, -52.01948904748973, -152..."
3,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,N/A,NA,N/A,EUR,7454,"[-244.0423763776673, 99.07380651498372, 136.02..."
4,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Stressed,Credit spread_Allegheny Energy_Implied spread,Credit spread,delta,EUR,7454,"[-1676.454801733736, -252.03985103138143, 909...."


### Book table

The book table will enrich the data model with information about the books that contain our trades.

In [4]:
books = pd.read_csv(
    "s3://data.atoti.io/notebooks/atoti-pandas-comparison/data/books.csv"
)
books.head()

,BOOKID,BOOK,TRADING_DESK,SUB_BUSINESS_UNIT,BUSINESS_UNIT
0,1,KE001,Keres,Domestic Rates & Credit,Rates & Credits
1,2,HE002,Hermes,Foreign Equity,Equity
2,3,HA003,Hades,Developed Market,Forex
3,4,HE004,Hermes,Domestic Rates & Credit,Rates & Credits
4,5,HE005,Hermes,Domestic Rates & Credit,Rates & Credits


### Merge tables

In [5]:
merged = trades.merge(books, how="left", on="BOOKID")

## Computing the VaR Metrics

Since pandas doesn't inherently manage dynamic aggregation, a pre-aggregation step necessarily needs to be coded for each desired level of aggregation before the statistical/mathematical function is applied to get the final metric.

The `groupby()` function is thus used to perform the initial aggregation of the PnL Vectors, and then the quantile/mean functions would be applied on the resulting set.

The downside of this is that each level of aggregation would require its own, multi-line code block, wich introduces **inefficiencies** and **redundancy**. 
In Atoti, however, a measure is configured only once, with one line of code, and can be subsequently used and evaluated at any granularity.

### Computing Top of House VaR and ES

In [6]:
%%time
%%memit

VaRTopOfHouse=merged.groupby(['ASOFDATE'])['PNL_VECTOR'].sum().reset_index()
VaRTopOfHouse['VaR95']=VaRTopOfHouse.apply(lambda x: np.quantile(x['PNL_VECTOR'], 0.05), axis=1)
VaRTopOfHouse['VaR99']=VaRTopOfHouse.apply(lambda x: np.quantile(x['PNL_VECTOR'], 0.01), axis=1)
VaRTopOfHouse['ES95']=VaRTopOfHouse.apply(lambda x: np.mean(sorted(x['PNL_VECTOR'][:12])), axis=1)
VaRTopOfHouse

peak memory: 9239.84 MiB, increment: 0.64 MiB
CPU times: user 940 ms, sys: 207 ms, total: 1.15 s
Wall time: 1.57 s


### Computing VaR and ES at BookId and TradeId Level

In [7]:
%%time
%%memit

VaRByBookandTrade=merged.groupby(['ASOFDATE', 'BOOK', 'TRADEID'])['PNL_VECTOR'].sum().reset_index()
VaRByBookandTrade['VaR95']=VaRByBookandTrade.apply(lambda x: np.quantile(x['PNL_VECTOR'], 0.05), axis=1)
VaRByBookandTrade['VaR99']=VaRByBookandTrade.apply(lambda x: np.quantile(x['PNL_VECTOR'], 0.01), axis=1)
VaRByBookandTrade['ES95']=VaRByBookandTrade.apply(lambda x: np.mean(sorted(x['PNL_VECTOR'][:12])), axis=1)
VaRByBookandTrade

peak memory: 10054.73 MiB, increment: 814.88 MiB
CPU times: user 20.5 s, sys: 262 ms, total: 20.8 s
Wall time: 21.3 s


### Computing VaR and ES at the most granular level (combination of all available qualitative hierarchies) 

In [8]:
%%time
%%memit

VaRGranular=merged.groupby(['ASOFDATE', 'BUSINESS_UNIT', 'SUB_BUSINESS_UNIT', 'TRADING_DESK' , 'BOOKID', 'RISKCLASS', 'TRADEID'])['PNL_VECTOR'].sum().reset_index()
VaRGranular['VaR95']=VaRGranular.apply(lambda x: np.quantile(x['PNL_VECTOR'], 0.05), axis=1)
VaRGranular['VaR99']=VaRGranular.apply(lambda x: np.quantile(x['PNL_VECTOR'], 0.01), axis=1)
VaRGranular['ES95']=VaRGranular.apply(lambda x: np.mean(sorted(x['PNL_VECTOR'][:12])), axis=1)
VaRGranular

peak memory: 12143.61 MiB, increment: 2122.67 MiB
CPU times: user 1min 6s, sys: 790 ms, total: 1min 7s
Wall time: 1min 8s


### Incremental VaR (Parent VAR - Parent VAR excluding self)

Since pandas does not have any semantic dimension attached to its columns, it lacks the understanding of any potential hierarchical/order relationships that may lie within them, which means that implementing a measure such as incremental VaR would require us to hard code every step of that logic.

In [9]:
%%time
%%memit

bookpnl=merged.groupby(['BOOK'])['PNL_VECTOR'].sum().reset_index()
deskpnl=merged.groupby(['TRADING_DESK'])['PNL_VECTOR'].sum().reset_index()
deskandbook=merged[['TRADING_DESK', 'BOOK']].drop_duplicates().sort_values(by=['TRADING_DESK', 'BOOK']).reset_index(drop=True)

deskandbook['DESKPNL']=''
deskandbook['BOOKPNL']=''

for i in range(len(deskandbook)):
    book=deskandbook.loc[i, 'BOOK']
    desk=deskandbook.loc[i, 'TRADING_DESK']
    bookpl=bookpnl[bookpnl['BOOK']==book]['PNL_VECTOR']
    deskpl=deskpnl[deskpnl['TRADING_DESK']==desk]['PNL_VECTOR']
    deskandbook.at[i, 'BOOKPNL']=bookpl.values
    deskandbook.at[i, 'DESKPNL']=deskpl.values
    
deskandbook['DESKWITHOUTBOOK']=deskandbook['DESKPNL']-deskandbook['BOOKPNL']
deskandbook['VaRDesk95']=deskandbook.apply(lambda x:  np.quantile(np.quantile(x['DESKPNL'], 0.05), 0.05), axis=1)
deskandbook['VaRDeskWITHOUTBOOK95']=deskandbook.apply(lambda x: np.quantile(x['DESKWITHOUTBOOK'], 0.05), axis=1)

deskandbook['IncrementalVaR']=deskandbook['VaRDesk95']-deskandbook['VaRDeskWITHOUTBOOK95']

peak memory: 11821.27 MiB, increment: 14.42 MiB
CPU times: user 1.82 s, sys: 131 ms, total: 1.95 s
Wall time: 2.53 s


In [10]:
deskandbook[["TRADING_DESK", "BOOK", "IncrementalVaR"]]

,TRADING_DESK,BOOK,IncrementalVaR
0,Apollo,AP016,-4.250068e+06
1,Apollo,AP023,-3.934346e+06
2,Apollo,AP028,-7.538250e+06
3,Atlas,AT030,-3.524092e+07
4,Hades,HA003,-1.412055e+07
5,Hades,HA012,-1.434995e+07
6,Hera,HE025,-1.063771e+07
7,Hera,HE026,-8.512933e+06
8,Heracles,HR006,-6.501539e+06
9,Heracles,HR014,-1.426574e+07


## What-if scenarios

pandas does not support branching either, which means that the only way to create something similar to a simulation or new scenario would require loading a full new dataset in a separate data structure, not only the deltas (differing data points). This doubles the memory footprint and limits any possibility of contained, side-by-side analytics.

We will start by making a copy of the original dataset, then enriching it with the stressed data points, then dropping the duplicates manually to remove the old, unstressed records.

In [11]:
stressed = merged.copy()

In [12]:
addition = pd.read_parquet(
    "data.atoti.io/notebooks/atoti-pandas-comparison/data/simulation/pnl_16.parquet",
    filesystem=s3,
)
stressed = pd.concat([stressed, addition])

In [13]:
stressed.drop_duplicates(
    ["BOOKID", "ASOFDATE", "TRADEID", "DATASET", "RISKFACTOR", "RISKCLASS"], keep="last"
)

,BOOKID,ASOFDATE,TRADEID,DATASET,RISKFACTOR,RISKCLASS,SENSITIVITYNAME,CCY,TID,PNL_VECTOR,BOOK,TRADING_DESK,SUB_BUSINESS_UNIT,BUSINESS_UNIT
0,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,Credit spread_Allegheny Energy_Implied spread,Credit spread,delta,EUR,7454,"[-1277.1016253973248, -170.05582820769533, 587...",KE001,Keres,Domestic Rates & Credit,Rates & Credits
1,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,Foreign exchange_USD_Spot price,Foreign exchange,delta,EUR,7454,"[26194.603468126214, -15146.454845596281, 2217...",KE001,Keres,Domestic Rates & Credit,Rates & Credits
2,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,Interest rate_USD.OIS_Implied yield,Interest rate,delta,EUR,7454,"[-125.09660412815069, -52.01948904748973, -152...",KE001,Keres,Domestic Rates & Credit,Rates & Credits
3,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Historical,N/A,NA,N/A,EUR,7454,"[-244.0423763776673, 99.07380651498372, 136.02...",KE001,Keres,Domestic Rates & Credit,Rates & Credits
4,1,2021-05-11,BOND_Allegheny Energy 915efcc3,Stressed,Credit spread_Allegheny Energy_Implied spread,Credit spread,delta,EUR,7454,"[-1676.454801733736, -252.03985103138143, 909....",KE001,Keres,Domestic Rates & Credit,Rates & Credits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94957,17,2021-05-11,SWAPTION_USD_CPI f1fe3379,Stressed,Foreign exchange_USD_Spot price,Foreign exchange,delta,EUR,48013,"[-3165.17284944808, 9784.91878288804, 12650.82...",NaN,NaN,NaN,NaN
94958,17,2021-05-11,SWAPTION_USD_CPI f1fe3379,Stressed,Interest rate_USD Vol_Implied volatility,Interest rate,vega,EUR,48013,"[1314.5518507700285, 2599.3993776308616, -264....",NaN,NaN,NaN,NaN
94959,17,2021-05-11,SWAPTION_USD_CPI f1fe3379,Stressed,Interest rate_USD.OIS_Implied yield,Interest rate,delta,EUR,48013,"[15.872083006760096, 38.95091031041502, -13.82...",NaN,NaN,NaN,NaN
94960,17,2021-05-11,SWAPTION_USD_CPI f1fe3379,Stressed,Interest rate_USD_CPI_Implied yield,Interest rate,delta,EUR,48013,"[745.9206026511285, 9628.152298063962, -476.38...",NaN,NaN,NaN,NaN


<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=atoti-pandas-comparison" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Your-turn-to-try-Atoti.jpg" alt="Try Atoti"></a></div>